## **Next-Generation Agricultural Market Forecasting and Insights System for Kenya**

### Business Understanding
### Business Overview

Kenya’s economic stability is intrinsically linked to the performance of its agricultural sector, which serves as both a primary GDP contributor and the backbone of livelihoods for millions of households. However, the sector continues to face structural inefficiencies characterized by persistent price volatility, fragmented market coordination, information asymmetry, and sentiment-driven demand fluctuations. These systemic challenges erode income predictability for farmers and traders, constrain value chain optimization, and ultimately weaken overall market resilience. In a data-driven global economy, the absence of real-time market intelligence and forward-looking analytics represents a critical strategic gap. Addressing this gap through an integrated, predictive market insights and forecasting system presents a transformative opportunity to enhance transparency, stabilize incomes, strengthen decision-making, and drive sustainable economic impact across Kenya’s agricultural ecosystem.
Stakeholders and Their Gains
Farmers (Small-Scale & Large-Scale)
Gains:
Informed decisions on when and where to sell their produce for maximum profit.
Insights into seasonal price trends to optimize crop production cycles.
Awareness of public sentiment that may impact market demand.


### Traders & Distributors (Wholesalers & Retailers)
#### Gains:
Forecasting tools to plan optimal buying and selling times.
Reduction in losses due to unexpected price drops.
Better logistics planning by analyzing regional price variations.

### Consumers (General Public)
#### Gains:
Awareness of expected price changes for household budgeting.
More stable prices due to better market efficiency.
Potential for lower food costs as market inefficiencies decrease.

### Problem Statement
Kenya’s agricultural markets exhibit pronounced price volatility across regions and commodity categories, creating significant uncertainty for market participants. These fluctuations are driven by seasonal production cycles, shifting demand patterns, regional supply imbalances, and evolving public perception. For example, staple commodities such as maize experience substantial temporal and geographic price disparities, directly influencing farmers’ commercialization strategies and revenue outcomes. Compounding these structural inefficiencies is the growing influence of digital public sentiment, particularly on social media platforms, which increasingly shapes market expectations and short-term demand behavior. The convergence of these factors results in a complex and opaque decision-making environment, limiting stakeholders’ ability to optimize pricing, distribution, and market timing strategies.

### Objectives
### Assess Commodity Price Fluctuations
* Evaluate the price volatility of various commodities across different regions and time periods
* Identify the factors contributing to price fluctuations, such as seasonal variations, market demand e.t.c.
### Analyze Market Trends:
* Monitor market trends and patterns in commodity prices using historical data and advanced analytics.
* Develop predictive models to forecast future market trends and price movements.


**DATA UNDERSTANDING**

The primary dataset contains Food Prices data for Kenya, sourced from the Kenya Agricultural Market Information System KAMIS, developed by the Ministry of Agriculture, Livestock, Fisheries, and Cooperatives. This database covers foods such as maize, rice, beans and beef, ,is updated daily(though mostly contains monthly data). The platform offers data on commodity prices, trade volumes, and market highlights

***Understanding rows and columns of our data***

In [1]:
#importing necessary libraries
import pandas as pd
import glob
import os
import warnings
import time
import sys
import matplotlib.pyplot as plt
import seaborn as sns
# importing custom scripts for different functions
sys.path.append('scripts')  # add scripts to path




# Ignore warnings
warnings.filterwarnings("ignore")

In [2]:
combined_df=pd.read_csv('../combined.csv')

In [3]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57010 entries, 0 to 57009
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Commodity       57010 non-null  object 
 1   Classification  57010 non-null  object 
 2   Grade           57010 non-null  object 
 3   Sex             57010 non-null  object 
 4   Market          57010 non-null  object 
 5   Wholesale       57010 non-null  object 
 6   Retail          57010 non-null  object 
 7   Supply Volume   39235 non-null  float64
 8   County          56991 non-null  object 
 9   Date            57010 non-null  object 
dtypes: float64(1), object(9)
memory usage: 4.3+ MB


In [4]:
combined_df.to_csv("combined.csv", index=False)

In [5]:
combined_df.isna().sum()

Commodity             0
Classification        0
Grade                 0
Sex                   0
Market                0
Wholesale             0
Retail                0
Supply Volume     17775
County               19
Date                  0
dtype: int64

In [6]:
combined_df.drop(columns=['Supply Volume'], axis=1, inplace=True)

In [7]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df.head()

,Commodity,Classification,Grade,Sex,Market,Wholesale,Retail,County,Date
0,Banana (Ripening),-,-,-,Mukuyu Market,-,50.00/Kg,Muranga,2026-02-11
1,Banana (Ripening),-,-,-,Mulot,70.00/Kg,100.00/Kg,Bomet,2026-02-11
2,Banana (Ripening),-,-,-,Kitale Municipality Market,80.00/Kg,100.00/Kg,Trans-Nzoia,2026-02-11
3,Banana (Ripening),-,-,-,Kerugoya,20.00/Kg,40.00/Kg,Kirinyaga,2026-02-10
4,Banana (Ripening),-,-,-,Bondeni,80.00/Kg,100.00/Kg,Trans-Nzoia,2026-02-10


In [8]:
data=combined_df.drop(columns=['Classification', 'Grade', 'Sex', 'Market'], axis=1)

In [9]:
for col in ['Retail', 'Wholesale']:
    data[col] = (
        data[col]
        .astype(str)
        .str.replace('/Kg', '', regex=False)
        .str.replace('-', '', regex=False)
        .str.strip()
    )


In [10]:
for col in ['Retail', 'Wholesale']:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57010 entries, 0 to 57009
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Commodity  57010 non-null  object        
 1   Wholesale  45298 non-null  float64       
 2   Retail     54612 non-null  float64       
 3   County     56991 non-null  object        
 4   Date       57010 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 2.2+ MB


In [11]:
# Aggregate
df_monthly = (
    data
    .groupby(['County', 'Commodity', pd.Grouper(key='Date', freq='MS')])
    .agg({'Retail': 'mean', 
          'Wholesale': 'mean'})
    .reset_index()
)
df_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6355 entries, 0 to 6354
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   County     6355 non-null   object        
 1   Commodity  6355 non-null   object        
 2   Date       6355 non-null   datetime64[ns]
 3   Retail     6302 non-null   float64       
 4   Wholesale  5731 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 248.4+ KB


In [12]:
def enforce_monthly_index(group):
    idx = pd.date_range(
        start=group['Date'].min(),
        end=group['Date'].max(),
        freq='MS'
    )
    group = (
        group
        .set_index('Date')
        .reindex(idx)
    )
    group['County'] = group['County'].iloc[0]
    group['Commodity'] = group['Commodity'].iloc[0]
    return group.reset_index().rename(columns={'index': 'Date'})


In [13]:
df_complete = (
    df_monthly
    .groupby(['County', 'Commodity'], group_keys=False)
    .apply(enforce_monthly_index)
)
df_complete.head()

,Date,County,Commodity,Retail,Wholesale
0,2023-05-01,Baringo,Cowpea leaves (Kunde),60.0,40.0
1,2023-06-01,Baringo,Cowpea leaves (Kunde),30.0,25.0
0,2022-12-01,Baringo,Omena,200.0,150.0
0,2022-10-01,Baringo,Red Irish potato,60.0,55.0
1,2022-11-01,Baringo,Red Irish potato,63.4,53.6


In [14]:
df_complete=df_complete.set_index('Date')
df_complete = df_complete.sort_index()
df_complete[['Retail', 'Wholesale']] = (
    df_complete
    .groupby(['County', 'Commodity'])[['Retail', 'Wholesale']]
    .apply(lambda x: x.interpolate(method='time'))
)

In [15]:
df_complete.reset_index()

,Date,County,Commodity,Retail,Wholesale
0,2022-02-01,Nyandarua,Wheat,56.000000,48.0000
1,2022-02-01,Kisii,Wheat,80.000000,60.0000
2,2022-02-01,Uasin-Gishu,Wheat,80.000000,50.0000
3,2022-02-01,Trans-Nzoia,Wheat,75.000000,58.8900
4,2022-02-01,Kirinyaga,Wheat,66.666667,57.5000
...,...,...,...,...,...
9184,2026-02-01,Trans-Nzoia,Beans (Yellow-Green),61.332500,49.0825
9185,2026-02-01,Meru,Wheat,90.000000,75.0000
9186,2026-02-01,Kirinyaga,Meat Beef,158.300000,137.0000
9187,2026-02-01,Kirinyaga,Dry Onions,31.500000,20.5000


## SARIMA

In [16]:
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf

In [17]:
def adf_pvalue(series):
    return adfuller(series.dropna(), autolag='AIC')[1]



def determine_d(series, alpha=0.05, max_d=2, min_obs=24):
    series = pd.Series(series).dropna()

    for d in range(max_d + 1):
        test_series = series.diff(d).dropna() if d > 0 else series

        if len(test_series) < min_obs:
            break

        pval = adf_pvalue(test_series)


        if not np.isnan(pval) and pval <= alpha:
            return d

    return min(1, max_d)


def determine_D(series, m=12):
    # Improved - use seasonal decomposition and statistical tests
    
    if len(series) < 2 * m:
        return 0
    
    try:
        # Check seasonal strength using decomposition
        decomposition = seasonal_decompose(series, model='additive', period=m)
        seasonal_strength = max(0, 1 - np.var(decomposition.resid) / np.var(series + decomposition.seasonal))
        
        # Also check ACF at seasonal lags
        acf_vals = acf(series.dropna(), nlags=m*2, fft=True)
        seasonal_acf = np.mean([abs(acf_vals[i]) for i in range(m, min(m*3, len(acf_vals)), m)])
        
        return 1 if seasonal_strength > 0.3 or seasonal_acf > 0.2 else 0
    except:
        return 0



In [18]:
# Improved - try multiple orders
def select_best_sarima_order(ts, d, D, seasonal_period=12):
    best_aic = np.inf
    best_order = None
    best_seasonal_order = None
    
    # Test different combinations
    for p in range(0, 3):
        for q in range(0, 3):
            for P in range(0, 2):
                for Q in range(0, 2):
                    try:
                        model = SARIMAX(
                            ts,
                            order=(p, d, q),
                            seasonal_order=(P, D, Q, seasonal_period),
                            enforce_stationarity=False,
                            enforce_invertibility=False
                        )
                        results = model.fit(disp=False, maxiter=200)
                        if results.aic < best_aic:
                            best_aic = results.aic
                            best_order = (p, d, q)
                            best_seasonal_order = (P, D, Q, seasonal_period)
                    except:
                        continue
    
    return best_order, best_seasonal_order

In [19]:
def backtest_sarima(ts, test_size=6):
    """Backtest SARIMA model"""
    train = ts[:-test_size]
    test = ts[-test_size:]
    
    # Fit on training data
    d = determine_d(train)
    D = determine_D(train)
    
    best_order, best_seasonal_order = select_best_sarima_order(train, d, D)
    
    model = SARIMAX(
        train,
        order=best_order,
        seasonal_order=best_seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    results = model.fit(disp=False)
    
    # Forecast
    forecast = results.forecast(steps=test_size)
    
    # Calculate error metrics
    mae = np.mean(np.abs(np.exp(forecast) - np.exp(test)))
    rmse = np.sqrt(np.mean((np.exp(forecast) - np.exp(test))**2))
    mape = np.mean(np.abs((np.exp(test) - np.exp(forecast)) / np.exp(test))) * 100
    
    return {'mae': mae, 'rmse': rmse, 'mape': mape}

In [20]:
def forecast_ci(results, steps=6, alpha=0.05):
    forecast = results.get_forecast(steps=steps)
    return forecast.summary_frame(alpha=alpha)

In [21]:
def improved_sarima_pipeline(df, price_col='Retail', date_col='Date', forecast_steps=6):
    models = {}
    forecasts = {}
    bs_results={}

    grouped = df.groupby(['Commodity'])
    
    for commodity, group in grouped:
        # Prepare time series
        # If Date is the index, reset it to make it a column
        if date_col in group.index.names or group.index.name == date_col:
            group = group.reset_index()
        ts = group.set_index(date_col)[price_col].resample('M').mean().dropna()

        # Need at least 2 years for seasonal modeling
        if len(ts) < 24:
            continue
        
        ts=np.log(ts)
        
        # Determine differencing
        d = determine_d(ts)
        D = determine_D(ts)
        
        # Select best SARIMA orders
        best_order, best_seasonal_order = select_best_sarima_order(
            ts, d, D
        )
        
        if best_order is None:
            continue
        
        # Fit final model
        model = SARIMAX(
            ts,
            order=best_order,
            seasonal_order=best_seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        results = model.fit(disp=False, maxiter=500)
        # Backtest
        backtest_results = backtest_sarima(ts, test_size=6)
            
        # Store results
        key = commodity
        models[key] = results
        # Generate forecasts
        forecast_df = forecast_ci(results, steps=forecast_steps)
        forecasts[key] = forecast_df
        bs_results[key] = backtest_results
    return models, forecasts, bs_results

In [22]:
retail_models, retail_forecasts, retail_error = improved_sarima_pipeline(
    df=df_complete,
    date_col='Date',
    price_col='Retail',
    forecast_steps=6
)

c:\Users\Victor\anaconda3\envs\learn-env\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Victor\anaconda3\envs\learn-env\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Victor\anaconda3\envs\learn-env\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Victor\anaconda3\envs\learn-env\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Victor\anaconda3\envs\learn

In [23]:
print(retail_models['Omena'].summary())

                                      SARIMAX Results                                       
Dep. Variable:                               Retail   No. Observations:                   40
Model:             SARIMAX(0, 1, 2)x(1, 1, [1], 12)   Log Likelihood                  -5.251
Date:                              Fri, 20 Feb 2026   AIC                             20.501
Time:                                      16:08:05   BIC                             22.926
Sample:                                  11-30-2022   HQIC                            19.604
                                       - 02-28-2026                                         
Covariance Type:                                opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -1.4686   6905.200     -0.000      1.000   -1.35e+04    1.35e+04
ma.L2          0.46

In [24]:
np.exp(retail_forecasts['Omena'])

Retail,mean,mean_se,mean_ci_lower,mean_ci_upper
2026-03-31,115.489928,1.398454,59.852012,222.848371
2026-04-30,85.814460,1.420614,43.123369,170.768698
2026-05-31,100.714319,1.420614,50.610826,200.419056
2026-06-30,112.143201,1.420614,56.354053,223.162255
2026-07-31,51.031100,1.420614,25.644081,101.550653
2026-08-31,75.308945,1.420614,37.844151,149.862977


In [25]:
wholesale_models, wholesale_forecasts, wholesale_error = improved_sarima_pipeline(
    df=df_complete,
    price_col='Wholesale',
    forecast_steps=6
)

c:\Users\Victor\anaconda3\envs\learn-env\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Victor\anaconda3\envs\learn-env\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Victor\anaconda3\envs\learn-env\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Victor\anaconda3\envs\learn-env\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Victor\anaconda3\envs\learn

In [26]:
print(wholesale_models['Omena'].summary())

                                     SARIMAX Results                                      
Dep. Variable:                          Wholesale   No. Observations:                   40
Model:             SARIMAX(0, 1, 1)x(0, 1, 1, 12)   Log Likelihood                 -10.075
Date:                            Fri, 20 Feb 2026   AIC                             26.150
Time:                                    16:11:44   BIC                             27.845
Sample:                                11-30-2022   HQIC                            25.801
                                     - 02-28-2026                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -1.0000   1.49e+04  -6.72e-05      1.000   -2.92e+04    2.92e+04
ma.S.L12      -0.1355      0.731   

In [27]:
len(wholesale_models)

13

In [28]:
np.exp(wholesale_forecasts['Omena'])

Wholesale,mean,mean_se,mean_ci_lower,mean_ci_upper
2026-03-31,117.941625,1.682815,42.524941,327.107493
2026-04-30,100.842078,1.682815,36.359542,279.682421
2026-05-31,54.405327,1.682815,19.616343,150.891512
2026-06-30,51.364487,1.682815,18.519940,142.457835
2026-07-31,153.108551,1.682815,55.204701,424.641887
2026-08-31,73.921187,1.682815,26.652966,205.018154
